In [ ]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
engine = create_engine("sqlite:///hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)

In [ ]:
Measurement = Base.classes.measurement
Station = Base.classes.station
session = Session(engine)

In [ ]:
precipitationlist = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > '2016-09-01').all()
precipitation = pd.DataFrame(precipitationlist)
precipitation = precipitation.set_index('date')
precipitation = precipitation.rename(columns={'prcp': 'precipitation'})
seaborn.set()
precipitation.plot(rot=45)
plt.show()

In [ ]:
precipitation.describe()

In [ ]:
station_count = session.query(func.count(Station.station)).limit(1).all()
station_count[0][0]

In [ ]:
session.query(Measurement.station, func.count(Measurement.id)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.id).desc()).all()

In [ ]:
session.query(Measurement.station, func.count(Measurement.id)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.id).desc()).limit(1).all()

In [ ]:
temperatureobservation = session.query(Measurement.date, Measurement.station, Measurement.tobs).filter(Measurement.date > '2018-10-29').all()
temperatureobservation

In [ ]:
tempobsforhighobsstat = session.query(Measurement.date, Measurement.tobs).filter(Measurement.date > '2018-10-29).filter(Measurement.station == 'USC00519281').all()
tempobsforhighobsstat

In [ ]:
tempobsforhighobsstatfreq = session.query(Measurement.date, Measurement.tobs).filter(Measurement.date > '2018-10-29').filter(Measurement.station == 'USC00519281').all()
tempobsfreq = pd.DataFrame(tempobsforhighobsstatfreq)
tempobsfreq = tempobsfreq.set_index("date")
seaborn.set()
tempobsfreq.plot.hist(bins=12)
plt.show()

In [ ]:
def calc_temps(start_date, end_date):
    min_temp = session.query(func.min(Measurement.tobs)).filter(Measurement.date > start_date).filter(Measurement.date < end_date)
    avg_temp = session.query(func.avg(Measurement.tobs)).filter(Measurement.date > start_date).filter(Measurement.date < end_date)
    max_temp = session.query(func.max(Measurement.tobs)).filter(Measurement.date > start_date).filter(Measurement.date < end_date)
    return min_temp, avg_temp, max_temp

In [ ]:
min_temp, avg_temp, max_temp = calc_temps("2018-01-01", "2018-12-31")

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(1.4, ((max_temp[0][0] - min_temp[0][0])/2) + min_temp[0][0], yerr=((max_temp[0][0] - min_temp[0][0])/2), color="black")

plt.title("Trip Avg Temp")
plt.ylabel("Temp (F)")
plt.ylim(0, 100)


plt.bar(1, avg_temp[0][0], facecolor="salmon", alpha=0.75, align="edge")
plt.xticks(avg_temp[0])

plt.show()